# Alignment hypergraph to variant graph with networkx and masked arrays

Convert alignment hypergraph to variant graph before visualization.

1. Create alignment hypergraph as a list of hyperedge objects
1. Convert alignment hypergraph to numpy masked array
1. Create variant graph from masked array using networkx
1. Create graphviz SVG output from variant graph
1. Other visualizations will be created from the variant graph (as in current CollateX)

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext memory_profiler

import warnings
warnings.filterwarnings('ignore')

import gc # garbage collection
import operator # comparison operators lt and gt
from collections import defaultdict, deque
from typing import Set, List
from dataclasses import dataclass
import networkx as nx
import re
import time
import queue

import numpy as np
import numpy.ma as ma

import graphviz
from IPython.display import SVG

import pprint
pp = pprint.PrettyPrinter(indent=2)

debug = None

## Load data

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks
print('aligning', how_many_paragraphs, 'paragraphs') # confirm

1 paragraphs from 6 witnesses


aligning 1 paragraphs


## Create data structures

1. Create witness_node (dataclass) for witness nodes. Properties (not all may be needed):
    1. token string: str
    1. witness: str
    1. offset in witness: int
    1. hyperedge: hyperedge instance (initially null)
    1. first_block: int (used for debugging, defaults to None)
1. Create alignment_hyperedge (dataclass) for … wait for it! … alignment hyperedges! Properties:
    1. witness_nodes: set of witness_node objects
1. Create lists for witness_node and alignment_hyperedge instances
1. Create START and END nodes and associated hyperedges

**TODO:** Can we abolish the alignment_hyperedge objects entirely, keeping all information in the vector space model?

In [3]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """TODO: Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int
    hyperedge: hyperedge = None
    # first_block: int = None # TODO: Remove this?

    def __repr__(self):
        return self.token_string

# Subclass of WitnessNodeEnd so that our constructed end node will be able to identify itself
class WitnessNodeEnd(WitnessNode):
    """TODO: Write a docstring some day"""

In [4]:
class AlignmentHyperedge():
    """TODO: Write a real docstring someday"""
    def __init__(self, kwargs):
        self.__dict__.update(kwargs) # overwrite the real value
        self._sigla = [key for key in kwargs.keys()]
    def __repr__(self):
        return "|".join(
            [":".join(
                    [str(key), str(getattr(self, key))]
                ) for key in self._sigla]
        )
    def __setitem__(self, key, value):
        # Value can be updated without adding another copy of the siglum to _sigla
        self.__dict__[key] = value
        if key not in self._sigla:
            self._sigla.append(key)
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness (keys will be sigla)
# create, for each witness:
#   nodes for real tokens
#   END node
# A start node is not created. That is on purpose.
for index, witness_siglum in enumerate(witness_sigla):
    # witness_sigla is a global set when the input data is read
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset))
    witness_node_lists[witness_siglum].append(WitnessNodeEnd('END', witness_siglum, len(token_array)))

In [6]:
def create_hyperedges(block):
    """Returns hyperedge data for block, together with offset_start_positions"""
    new_hyperedges = []
    for token_offset_in_block in range(block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in block.all_start_positions]
        tokens_to_place = [
            (
                witness_sigla[token_membership_array[token_array_index]],
                token_witness_offset_array[token_array_index]
            )
            for token_array_index in offset_start_positions
        ]
        dictionary = { # data for new hyperedge
            siglum: witness_node_lists[siglum][offset]
            for siglum, offset in tokens_to_place
        }
        # then create an AlignmentHyperedge instance for the new hyperedge
        #   and add to list of alignment hyperedges
        hyperedge = AlignmentHyperedge(dictionary)
        new_hyperedges.append(hyperedge)

    return (new_hyperedges)

In [7]:
def convert_hyperedge_to_masked_array(hyperedge, sigla: list):
    """Converts hyperedge to masked array

    hyperedge: hyperedge to convert
        (list of witness nodes, with witness as key and witness offset as value)
    witnesses: list of sigla in fixed order

    Returns: masked array with values masked for witnesses not in hyperedge
    """
    row = []
    mask = []
    for siglum in sigla:
        if siglum in hyperedge: # enter witness offset and set mask to False
            row.append(hyperedge[siglum].witness_offset)
            mask.append(False)
        else:
            row.append(np.nan)
            mask.append(True)
    return ma.MaskedArray(row, mask=mask)

In [8]:
def convert_alignment_graph_to_numpy_masked_array(existing_ag, witness_sigla: list):
    """Create numpy masked array (matrix) from alignment graph

    Horizontal dimension = number of witnesses (global)
    Vertical dimension = number of hyperedges in alignment hypergraph except START and END

    witnesses : list of sigla (global)

    Returns: two-dimensional masked array, with values not in hyperedges masked
    """
    rows = [] # each row is a masked array, one per hyperedge
    for hyperedge in existing_ag:
        rows.append(convert_hyperedge_to_masked_array(hyperedge, witness_sigla)) # each hyperedge is a masked array
    return ma.MaskedArray(rows) # outer dimension must be masked for inner mask to be seen

In [9]:
def check_whether_okay_to_place(current_hyperedges, potential_hyperedge, debug=None) -> bool:
    """Return True iff we can add row without creating crossing hyperedge

    current_hyperedge: two-dimensional output of convert_alignment_graph_to_numpy_masked_array()
    potential_hyperedge: convert here to masked array

    If subtracting a potential hyperedge from any existing hyperedge would
    return values that diverge in sign, the potential would cross that existing"""
    current_hyperedges_as_ma = convert_alignment_graph_to_numpy_masked_array(current_hyperedges, witness_sigla)
    potential_hyperedge_as_ma = convert_hyperedge_to_masked_array(potential_hyperedge, witness_sigla)
    subtractionResult = current_hyperedges_as_ma - potential_hyperedge_as_ma
    signs = np.sign(subtractionResult)
    okayToPlace = (signs.min(axis=1) == signs.max(axis=1)).all()
    if debug:
        print(f"{current_hyperedges_as_ma=}")
        print(f"{potential_hyperedge_as_ma=}")
        print(f"{subtractionResult=}")
        print(f"{signs=}")
        print(f"{signs.min(axis = 1)}")
        print(f"{signs.max(axis = 1)}")
        print(f"{okayToPlace=}")
        print(f"{type(okayToPlace)=}")
    return True if okayToPlace is ma.masked else okayToPlace

In [10]:
def merge_hyperedge(potential_hyperedge, debug=False):
    # check whether a witness_node has got the hyperedge property set
    # a hyperedge may contain zero, some, or all witnesses; we need to handle all these cases
    found = set([witnessnode.hyperedge for siglum, witnessnode in potential_hyperedge.items() if witnessnode.hyperedge])
    if len(found) > 1:
        # found two or more existing hyperedges...
        # remove them from the hyperedges set
        for existing_hyperedge in found:
            if debug==True:
                print("We are trying to remove the hyperedge: "+str(existing_hyperedge))
                print("Existing hyperedge has nodes: " + str(existing_hyperedge))
                print("Potential hyperedge has nodes: " + str(potential_hyperedge))
            hyperedges.remove(existing_hyperedge)
            # NB: Siglum may be in both existing and potential; we assume (!) always
            #   with the same value
            # TODO (maybe): Should we override the keys method in Alignment_Hyperedge instead of checking _sigla?
            for siglum in existing_hyperedge._sigla: # don't get all keys, since that would include _sigla
                if siglum not in potential_hyperedge:
                    potential_hyperedge[siglum] = existing_hyperedge[siglum]
        # replace them by a new hyperedge
        hyperedges.append(potential_hyperedge)
        # update hyperedge property on the witness nodes involved in the hyperedge
        new_hyperedge = potential_hyperedge
        for witness_node in new_hyperedge.values():
            witness_node.hyperedge = new_hyperedge

    elif len(found) == 0:
        # we checked for transpositions
        # TODO: Can we filter out hyperedges we've already placed, so that we don't have to process them
        #   fully each time
        hyperedges.append(potential_hyperedge)
        # update hyperedge property on the witness nodes involved in the hyperedge
        new_hyperedge = potential_hyperedge
        for witness_node in new_hyperedge.values():
            witness_node.hyperedge = new_hyperedge
    else: # update single existing hyperedge
        for siglum in potential_hyperedge._sigla:
            list(found)[0][siglum] = potential_hyperedge[siglum] # add node to hyperedge (possibly redundantly)
            potential_hyperedge[siglum].hyperedge = list(found)[0] # and update hyperedge property of node

In [11]:
def process_blocks(selected_blocks):
    for index, selected_block in enumerate(selected_blocks):
        # Create hyperedges for entire block
        new_hyperedges = create_hyperedges(selected_block)
        # Check only first and last, merge is both are okay
        # NB: Checking only first and last to test entire block will break with discontinuous blocks
        #   (should we ever switch to working with discontinuous blocks)
        if index == 0:
            # add the first block without checking for transposition since matrix will be empty
            for potential_hyperedge in new_hyperedges:
                merge_hyperedge(potential_hyperedge)
        else:
            debug_flag = None
            no_transposition = check_whether_okay_to_place(hyperedges, new_hyperedges[0], debug_flag) and check_whether_okay_to_place(hyperedges, new_hyperedges[-1])
            if no_transposition: # all-or-nothing
                if debug:
                    print("Block #" + str(index) + " has no transposition")
                for potential_hyperedge in new_hyperedges:
                    merge_hyperedge(potential_hyperedge, debug=False)
            else:
                if debug:
                    print("Block #"+ str(index) + " (allegedly) has a transposition")

In [12]:
# create all hyperedges (gingerly)
# We exclude blocks with repetition (temporarily?)
selected_blocks = filter(lambda x: len(x.all_start_positions) == x.witness_count, prioritized_blocks)

# store all hyperedges in list
hyperedges = []

# %prun profiles; see https://towardsdatascience.com/magic-commands-for-profiling-in-jupyter-notebook-d2ef00e29a63
# %prun process_blocks(selected_blocks)
process_blocks(selected_blocks)

## Convert alignment graph to numpy masked array

In [13]:
vector_space_as_ma = convert_alignment_graph_to_numpy_masked_array(hyperedges, witness_sigla)

## Create variant graph from alignment hypergraph

First create variant graph nodes. Walk over witness nodes in `witness_node_lists`. Keep track of which hyperedges have been added to variant graph. For each witness node:

1. If node has no hyperedge, create variant graph node with selected information from witness node on it.
1. If node has new hyperedge, create variant graph node with selected information from all witness nodes from hyperedge on it.
1. If node has hyperedge that is already on variant graph, do nothing.

Witness nodes contain token string, a witness siglum, a witness offset, and an optional hyperedge. The hyperedge contains, as dictionary items, all witness sigla as keys with witness nodes as values. From this information, we put onto the variant graph node the token string plus the hyperedge dictionary with sigla as keys and witness offsets as values.

**Notes:**

* The variant graph does not need to contain the token string, since we can look it up later, but for ease of visualization we include it here.
* In Real Life CollateX a variant graph node contains properties other than the token string. We ignore that structure in our experiment as we concentrate on visualization.

In [14]:
class VG_node():
    """Variant graph node"""
    def __init__(self, token_string, data): # dictionary of siglum:witness_offset
        self.token_string = token_string
        self.__dict__.update(**data)
        self._sigla = [key for key in data.keys()]
    def __repr__(self):
        return self.token_string + "~" + "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [15]:
# Create networkx variant graph from our (not networkx) alignment graph
# Create variant graph edges for start and end nodes (only)
# We've treated VG as a global and we refer to it in the earlier functions; should we pass it instead?
def augment_or_add_edge_without_conversion(siglum, source_VG_node, target_VG_node):
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
        )

In [16]:
# Use alignment graph node to look up variant graph node
def alignment_node_to_VG_node(alignment_node: WitnessNode) -> VG_node:
    global witness_offset_to_VG_node
    return witness_offset_to_VG_node[alignment_node.witness][alignment_node.witness_offset]

In [17]:
# Create variant graph data edges (except start and end nodes)
def augment_or_add_edge(siglum, source, target):
    source_VG_node = alignment_node_to_VG_node(source)
    target_VG_node = alignment_node_to_VG_node(target)
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
                )

In [18]:
def create_variant_graph_from_vector_space():
    # create variant graph and add start and end nodes

    start_node = VG_node("START", {})
    end_node = VG_node("END", {})
    global VG # does this have to be global; we return it at the end, but we also use it in functions above
    VG = nx.DiGraph(start = start_node, end = end_node) # create start and end properties to find terminal nodes
    VG.add_node(start_node)
    VG.add_node(end_node)

    # keep track of which witness nodes (and therefore also hyperedges)
    # have been added to VG
    # does this duplicate information available from the following structure, since
    #   if we assign a value below to replace the None, this doesn't seem to add anything
    #   Can we get rid of this and use witness_offset_to_VG_node for this purpose instead?
    # NB: Includes START and END, which we will later ignore
    from bitarray import bitarray
    global VG_tracking # keep this, but should we return it instead of making it global?
    VG_tracking = {}
    for siglum, witness_node_list in witness_node_lists.items():
        VG_tracking[siglum] = bitarray(len(witness_node_list))
        VG_tracking[siglum].setall(0)

    # map from witness token node to variant graph node (needed to construct edges)
    # values will be added as we create variant graph nodes
    # NB: Includes START and END, which we will later ignore
    global witness_offset_to_VG_node # does this have to be global?
    witness_offset_to_VG_node = {}
    for siglum in witness_node_lists.keys():
        witness_offset_to_VG_node[siglum] = [None] * (len(witness_node_lists[siglum]))

    # Replaces old code in following cell to build variant graph,
    #   this time using vector space as main data source

    # 1. Convert vectors to variant graph nodes

    for row in vector_space_as_ma:
        if debug:
            print(' '.join(('Processing', str(row))))
        # create dictionary of siglum:value for node (variable name: data) and update globals
        data = {}
        for index, value in enumerate(row):
            if value != vector_space_as_ma.fill_value:
                siglum = 'w' + str(index)
                value = int(value)
                data[siglum] = value # add to eventually data for new VG node
                VG_tracking[siglum][value] = 1 # update global; do we need to adjust by 1?
        # get token string for node (variable name: token_string)
        siglum, offset = next(iter(data.items()))
        token_string = witness_node_lists[siglum][int(offset)].token_string
        # create and add new VG_node
        new_VG_node = VG_node(token_string, data)
        VG.add_node(new_VG_node)
        for siglum, offset in data.items(): # Eek! Another for loop! How embarrassing!
            witness_offset_to_VG_node[siglum][offset] = new_VG_node # update other global;

    # 2. Traverse variant graph (in arbitrary order), each of which contains witness nodes.
    #    Draw outgoing edges, which point to nodes with next tokens in each witness present
    #      on the variant graph node.
    #    Creating new variant graph nodes for witness tokens not in a hyperedge.
    #    Use queue because we're iterating over a dynamic structure (inventory of variant graph nodes).

    VG_node_queue = queue.Queue()
    #
    # Temporarily skipping START and END, which are the first two nodes
    #
    for node in VG.nodes(): # add all initial VG nodes to queue except START and END
        VG_node_queue.put(node)
        if debug:
            print(f"Adding node to VG_node_queue; length is {VG_node_queue.qsize()=}")
    ignore_start = VG_node_queue.get() # temporarily ignore START node when creating edges
    ignore_end = VG_node_queue.get() # temporarily ignore END node when creating edges
    if debug:
        print(f"Removed START and END; length is {VG_node_queue.qsize()=}")
    while not VG_node_queue.empty():
        if debug:
            print(f"Processing source node from VG_node_queue; length is {VG_node_queue.qsize()=}")
        source_node = VG_node_queue.get()
        targets = set()
        edge_labels = defaultdict(list) # key is target VG node, value is list of sigla
        sigla = source_node._sigla # all sigla on source node
        for siglum in sigla:
            # target may or may not already exist as node in VG
            source_offset = source_node[siglum]
            target_offset = source_offset + 1
            target = witness_offset_to_VG_node[siglum][target_offset]
            if target: # does the target already exist?:
                targets.add(target)
                edge_labels[target].append(siglum)
            else: # 1) create target, 2) add to targets, 3) add to queue of nodes, and 4 + 5) update globals
                # TODO: Do we need to update globals? We shouldn't need to return to nodes we add here.
                # NB: If real target object is of type WitnessNodeEnd, don't add it to the queue or create an edge
                # print(f"{type(witness_node_lists[siglum][target_offset])=}")
                witness_node_target = witness_node_lists[siglum][target_offset]
                if type(witness_node_target) != WitnessNodeEnd:
                    target_token_string = witness_node_target.token_string
                    new_VG_node = VG_node(target_token_string, {siglum: target_offset}) # 1 create target
                    targets.add(new_VG_node) # 2 add to targets
                    edge_labels[new_VG_node].append(siglum)
                    VG_node_queue.put(new_VG_node) # 3 add to queue of nodes
                    VG_tracking[siglum][target_offset] = 1 # 4 update first global
                    witness_offset_to_VG_node[siglum][target_offset] = new_VG_node # 5. update second global
        for target in targets:
            VG.add_edge(source_node, target, label=",".join(edge_labels[target]))
        #
        # Add edges for start VG nodes
        #
        all_first_data_nodes = defaultdict(list)
        for siglum in witness_sigla:
            key =  witness_offset_to_VG_node[siglum][0]
            all_first_data_nodes[key].append(siglum)
        for key,value in all_first_data_nodes.items():
            VG.add_edge(start_node, key, label=",".join(value))

        #
        # Add edges for end VG node
        #
        all_end_data_nodes = defaultdict(list)
        for siglum in witness_sigla:
            # Figure out why -2 works and clean up as needed
            key = witness_offset_to_VG_node[siglum][-2]
            all_end_data_nodes[key].append(siglum)
        for key,value in all_end_data_nodes.items():
            VG.add_edge(key, end_node, label=",".join(value))

    # we're done! return the result
    return VG

In [19]:
# # recover some memory
# del witness_node_list
# del witness_offset_to_VG_node
# del VG_tracking
gc.collect()
VG = create_variant_graph_from_vector_space()

0

## Visualize variant graph

In [20]:
# # node id values must be strings for graphviz
a = graphviz.Digraph(format="svg", name="variant_graph_unjoined")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG.nodes()):
    node_id = str(index)
    node_to_id[node] = node_id
    node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for edge in VG.edges(data=True):
    # edge is a three-item tuple: source, target, dictionary of properties
    a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=edge[2]["label"])

# print('aligning', how_many_paragraphs, 'paragraphs') # confirm
SVG(a.view())

In [21]:
#  This function joins the variant graph in place.
#  This function is a straight port of the Java version of CollateX.

def join(graph):
    processed = set()
    end = graph.graph["end"]
    queue = deque()
    for (_, neighbor) in graph.out_edges(graph.graph["start"]):
        queue.appendleft(neighbor)
    while queue:
        vertex = queue.popleft()
        out_edges = graph.out_edges(vertex)
        if len(out_edges) == 1:
            (_, join_candidate) = next(iter(out_edges))
            can_join = join_candidate != end and len(graph.in_edges(join_candidate)) == 1
            if can_join:
                join_vertex_and_join_candidate(graph, join_candidate, vertex)
                # we have merged join_candidate (tokens originally to the right) into vertex (tokens originally to the left)
                # (for now, join_candidate node and all of its edges are still there, and we need to remove edges before
                #   we can remove node, which we do in a for loop)
                #
                # RESUME HERE: both of the following branches are wrong, the first cryptically (it shows sigla, but not
                #   all sigla) and the second conspicuously
                #
                for (_, neighbor, data) in list(graph.out_edges(join_candidate, data=True)):
                    graph.remove_edge(join_candidate, neighbor)
                    if graph.has_edge(vertex, neighbor):
                        continue # TODO: this looks wrong
#                         graph[source][target]["siglum"].append(data["siglum"])
                    else:
#                         graph.add_edge(vertex, neighbor, siglum=data["siglum"])
                         graph.add_edge(vertex, neighbor,label=data["label"])
                graph.remove_edge(vertex, join_candidate)
                graph.remove_node(join_candidate)
                queue.appendleft(vertex)
                continue
        processed.add(vertex)
        for (_, neighbor) in out_edges:
            # FIXME: Why do we run out of memory in some cases here, if this is not checked?
            if neighbor not in processed:
                queue.appendleft(neighbor)


def join_vertex_and_join_candidate(graph, join_candidate, vertex):
    # Note: since there is no normalized/non normalized content in the graph
    # a space character is added here for non punctuation tokens

    if re.match(r'^\W', join_candidate.token_string):
        vertex.token_string += join_candidate.token_string
    else:
        vertex.token_string += (" " + join_candidate.token_string)
    # join_candidate must have exactly one token (inside a list); left item may have more
#     for siglum, token in join_candidate.tokens.items():
#         vertex.add_token(siglum, token[0])

In [22]:
# len(nx.algorithms.cycles.find_cycle(VG))

In [23]:
# nx.algorithms.cycles.find_cycle(VG)

In [24]:
join(VG)

In [25]:
# pp.pprint([edge for edge in VG.edges()])

In [26]:
## node id values must be strings for graphviz
a = graphviz.Digraph(format="svg", name="variant_graph_joined")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG.nodes()):
    node_id = str(index)
    node_to_id[node] = node_id
    node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for edge in VG.edges(data=True):
    # edge is a three-item tuple: source, target, dictionary of properties
#     label = "(all)" if len(edge[2]["siglum"]) == len(witness_sigla) else ",".join(sorted(edge[2]["siglum"]))
    label = edge[2]["label"]
    a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=label)

print('aligning', how_many_paragraphs, 'paragraphs') # confirm
SVG(a.view())

aligning 1 paragraphs


In [27]:
for edge in VG.edges(data=True):
    print(edge)

Unable to init server: Could not connect: Connection refused

(eog:747): Gtk-WARNING **: 21:56:37.554: cannot open display: :0


(START~, WHEN we~w0:0|w1:0|w2:0|w3:3|w4:3|w5:3, {'label': 'w0,w1,w2'})
(START~, Causes of Variability.~w3:0|w4:0|w5:0, {'label': 'w3,w4,w5'})
(we reflect on the vast diversity of the plants and animals which have been cultivated, and which have varied during all ages under the most different climates and treatment,~w0:53|w1:52|w2:52|w3:55|w4:55|w5:55, we are driven to conclude that this~w0:84|w1:83|w2:83|w3:86|w4:84|w5:84, {'label': 'w4,w5'})
(we reflect on the vast diversity of the plants and animals which have been cultivated, and which have varied during all ages under the most different climates and treatment,~w0:53|w1:52|w2:52|w3:55|w4:55|w5:55, I think~w0:82|w1:81|w2:81|w3:84, {'label': 'w0,w1,w2,w3'})
(due to our domestic productions having been raised under conditions of life not so uniform as, and somewhat different from, those to which the parent-species~w0:95|w1:94|w2:94|w3:97|w4:94|w5:94, had~w4:119|w5:119, {'label': 'w4,w5'})
(due to our domestic productions having been ra

In [28]:
# nx.algorithms.cycles.find_cycle(VG)

# To do next

1. Move SVG/Graphviz code into function
1. Create alignment table visualization without joining
1. Add joining to the alignment table (to check accuracy of joining results)
1. Edge labels are wrong, which may be a problem with the join() function
1. Test intermediate data sets, larger than a paragraph and smaller than a chapter
1. Reassess method of prioritizing blocks
1. Implement decision tree / graph

In [29]:
hyperedges

[w0:we|w1:we|w2:we|w3:we|w4:we|w5:we,
 w0:reflect|w1:reflect|w2:reflect|w3:reflect|w4:reflect|w5:reflect,
 w0:on|w1:on|w2:on|w3:on|w4:on|w5:on,
 w0:the|w1:the|w2:the|w3:the|w4:the|w5:the,
 w0:vast|w1:vast|w2:vast|w3:vast|w4:vast|w5:vast,
 w0:diversity|w1:diversity|w2:diversity|w3:diversity|w4:diversity|w5:diversity,
 w0:of|w1:of|w2:of|w3:of|w4:of|w5:of,
 w0:the|w1:the|w2:the|w3:the|w4:the|w5:the,
 w0:plants|w1:plants|w2:plants|w3:plants|w4:plants|w5:plants,
 w0:and|w1:and|w2:and|w3:and|w4:and|w5:and,
 w0:animals|w1:animals|w2:animals|w3:animals|w4:animals|w5:animals,
 w0:which|w1:which|w2:which|w3:which|w4:which|w5:which,
 w0:have|w1:have|w2:have|w3:have|w4:have|w5:have,
 w0:been|w1:been|w2:been|w3:been|w4:been|w5:been,
 w0:cultivated,|w1:cultivated,|w2:cultivated,|w3:cultivated,|w4:cultivated,|w5:cultivated,,
 w0:and|w1:and|w2:and|w3:and|w4:and|w5:and,
 w0:which|w1:which|w2:which|w3:which|w4:which|w5:which,
 w0:have|w1:have|w2:have|w3:have|w4:have|w5:have,
 w0:varied|w1:varied|w2:vari

In [30]:
# len(hyperedges)

In [31]:
# vector_space_as_ma.shape

In [32]:
vector_space_as_ma

masked_array(
  data=[[53.0, 52.0, 52.0, 55.0, 55.0, 55.0],
        [54.0, 53.0, 53.0, 56.0, 56.0, 56.0],
        [55.0, 54.0, 54.0, 57.0, 57.0, 57.0],
        [56.0, 55.0, 55.0, 58.0, 58.0, 58.0],
        [57.0, 56.0, 56.0, 59.0, 59.0, 59.0],
        [58.0, 57.0, 57.0, 60.0, 60.0, 60.0],
        [59.0, 58.0, 58.0, 61.0, 61.0, 61.0],
        [60.0, 59.0, 59.0, 62.0, 62.0, 62.0],
        [61.0, 60.0, 60.0, 63.0, 63.0, 63.0],
        [62.0, 61.0, 61.0, 64.0, 64.0, 64.0],
        [63.0, 62.0, 62.0, 65.0, 65.0, 65.0],
        [64.0, 63.0, 63.0, 66.0, 66.0, 66.0],
        [65.0, 64.0, 64.0, 67.0, 67.0, 67.0],
        [66.0, 65.0, 65.0, 68.0, 68.0, 68.0],
        [67.0, 66.0, 66.0, 69.0, 69.0, 69.0],
        [68.0, 67.0, 67.0, 70.0, 70.0, 70.0],
        [69.0, 68.0, 68.0, 71.0, 71.0, 71.0],
        [70.0, 69.0, 69.0, 72.0, 72.0, 72.0],
        [71.0, 70.0, 70.0, 73.0, 73.0, 73.0],
        [72.0, 71.0, 71.0, 74.0, 74.0, 74.0],
        [73.0, 72.0, 72.0, 75.0, 75.0, 75.0],
        [74.0, 73.0,

In [33]:
for node in VG.nodes:
    pp.pprint(node)

START~
END~
we reflect on the vast diversity of the plants and animals which have been cultivated, and which have varied during all ages under the most different climates and treatment,~w0:53|w1:52|w2:52|w3:55|w4:55|w5:55
due to our domestic productions having been raised under conditions of life not so uniform as, and somewhat different from, those to which the parent-species~w0:95|w1:94|w2:94|w3:97|w4:94|w5:94
WHEN we~w0:0|w1:0|w2:0|w3:3|w4:3|w5:3
look to~w0:2|w1:2|w2:2|w3:5
the individuals of the same variety or sub-variety of our older cultivated plants and animals, one of the first points which strikes~w0:4|w1:4|w2:4|w3:7|w4:6|w5:6
us,~w0:26|w1:26|w2:26|w3:29
is, that they generally differ~w0:27|w1:27|w2:27|w3:30|w4:29|w5:29
more~w0:33|w1:32|w2:32|w3:35|w5:34
from each~w0:34|w1:33|w2:33|w3:36|w4:34|w5:35
other~w1:35|w2:35|w3:38|w4:36|w5:37
than do the individuals of any one species or variety in a state of nature.~w0:37|w1:36|w2:36|w3:39|w4:38|w5:38
When~w0:52|w1:51|w2:51|w3:54
I 

In [34]:
# pp.pprint(witness_offset_to_VG_node)

In [35]:
token_membership_array

Unable to init server: Could not connect: Connection refused

(eog:855): Gtk-WARNING **: 21:56:39.866: cannot open display: :0


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 ' #1 ',
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,